# Average Undergraduate Tuition by State 2013-2021

<i><small>*Note from the author: While there are some basic visualizations to help convey findings, there is an associated dashboard with more detailed and in-depth visualizations available [here](https://public.tableau.com/app/profile/bradley.gauss/viz/wljkefnojwvbekjwafcnaslkeVGQ3OWE/RisingCostofCollege2013-2021).</small></i>

### Table of Contents

* [Goals](#Goals)
* [Data](#Data)
    * [Loading the Data](#Loading-the-Data)
    * [Preview the Data](#Preview-the-Data)
    * [Data Information](#Data-Information)
* [Cleaning the Data](#Cleaning-the-Data)
* [Data Analysis](#Data-Analysis)
    * [Tuition Change Trends](#Tuition-Change-Trends)
    * [Tuition by State](#Tuition-by-State)
* [Conclusions](#Conclusions)

### Goals
This notebook contains an exploratory analysis on the average cost of undergraduate education in the United States from 2013-2021. The goals for this project include:

* Reveal trends in the change of costs over time
* Compare costs between public and private institutions
* Compare costs among states

### Data
This data was downlaoded from Kaggle:

[Average cost of undergraduate student by state USA](https://www.kaggle.com/datasets/bhargavchirumamilla/average-cost-of-undergraduate-student-by-state-usa)

Further details on this dataset can be found by following the link above.

#### Loading the Data
This analysis will use the Pandas and SQLite libraries. A database connection is made. The data is loaded into a DataFrame called `costs`, and a preview of the DataFrame is printed. The DataFrame is pushed to the database for analysis using SQL.

In [2]:
# import required libraries
import pandas as pd
import sqlite3

In [3]:
# create database connection
conn = sqlite3.connect('my_jupyter.db')

In [4]:
# load sql extension
%load_ext sql

In [5]:
# connect to sql database
%sql sqlite:///my_jupyter.db

In [6]:
# load the data
cost_breakdown = pd.read_csv('edu_costs.csv')

# preview the data
cost_breakdown.head()

,Year,State,Type,Length,Expense,Value
0,2013,Alabama,Private,4-year,Fees/Tuition,13983
1,2013,Alabama,Private,4-year,Room/Board,8503
2,2013,Alabama,Public In-State,2-year,Fees/Tuition,4048
3,2013,Alabama,Public In-State,4-year,Fees/Tuition,8073
4,2013,Alabama,Public In-State,4-year,Room/Board,8473


In addition to expense types, this analysis is also interested in the total cost of attendance. A new DataFrame will be created that contains an aggregated column showing the total cost of attendance for each group. It groups the data by `Year`, `State`, `Type`, and `Length`. An aggregate column that combines costs of fees/tuition and room/board is created.

In [7]:
# group the data
costs = cost_breakdown.groupby(['Year', 'State', 'Type', 'Length']).agg({'Value': 'sum'}).reset_index()
costs.rename(columns={'Value': 'Total Cost'}, inplace=True)

In [8]:
costs.head()

,Year,State,Type,Length,Total Cost
0,2013,Alabama,Private,4-year,22486
1,2013,Alabama,Public In-State,2-year,4048
2,2013,Alabama,Public In-State,4-year,16546
3,2013,Alabama,Public Out-of-State,2-year,7736
4,2013,Alabama,Public Out-of-State,4-year,28853


#### Preview the Data
We need to understand the data we are working with:
* Understand columns and their datatypes
* Check for missing data
* Check shape of data

In [219]:
costs.dtypes

Year           int64
State         object
Type          object
Length        object
Total Cost     int64
dtype: object

In [220]:
# check shape of data
costs.shape

(2248, 5)

In [221]:
# check for missing data
costs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2248 entries, 0 to 2247
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Year        2248 non-null   int64 
 1   State       2248 non-null   object
 2   Type        2248 non-null   object
 3   Length      2248 non-null   object
 4   Total Cost  2248 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 87.9+ KB


#### Data Information
A quick inspection of the data revealed:
* There are 5 columns and 2248 rows
* There is no missing data
* The data contains objects and int datatypes

### Cleaning the Data
The data is relatively clean and ready for analysis. There are only two things to be first:
* change the `Year` datatype from int to object (it is categorical)
* change column names to lowercase and snake case format

In [222]:
# change the year column dtype form int to object
costs['Year'] = costs['Year'].astype('object')

In [223]:
# verfiy the change
costs.dtypes

Year          object
State         object
Type          object
Length        object
Total Cost     int64
dtype: object

In [239]:
# rename columns using a lambda regex function
costs = costs.rename(columns=lambda x: x.lower().replace(' ', '_'))

In [240]:
# verify the renaming has been done properly
costs.head()

,year,state,type,length,total_cost
0,2013,Alabama,Private,4-year,22486
1,2013,Alabama,Public In-State,2-year,4048
2,2013,Alabama,Public In-State,4-year,16546
3,2013,Alabama,Public Out-of-State,2-year,7736
4,2013,Alabama,Public Out-of-State,4-year,28853


### Data Analysis
The data is ready for analysis. The analysis is done in 2 phases. The first indentifies trends in tuition cost changes among several variables. The second analyzes costs by state.

The DataFrame is pushed to the database for SQL access.

#### Tuition Change Trends
Questions this portion of analysis answers:

What is the trend of tuition costs:
* at all institutions from 2013-2021?
* at private institutions from 2013-2021?
* at public institutions from 2013-2021?
* at public 2-year institutions from 2013-2021?
* at public 4-year institutions from 2013-2021?

In [241]:
# push costs df to database
costs.to_sql('costs', conn)

2248

Print a preview of the table used for this portion of analysis.

In [242]:
%%sql
SELECT *
FROM costs
LIMIT 5;

   sqlite:///'costs.db'
   sqlite:///'jupyter_sqlite.db'
 * sqlite:///my_jupyter.db
Done.


index,year,state,type,length,total_cost
0,2013,Alabama,Private,4-year,22486
1,2013,Alabama,Public In-State,2-year,4048
2,2013,Alabama,Public In-State,4-year,16546
3,2013,Alabama,Public Out-of-State,2-year,7736
4,2013,Alabama,Public Out-of-State,4-year,28853


<b>Question:</b> What is the trend of tuition costs at all institutions from 2013 to 2021?

In [261]:
%%sql
SELECT
    year, 
    ROUND(AVG(total_cost)) AS avg_total_cost,
    ROUND(((AVG(total_cost) - LAG(AVG(total_cost)) OVER (ORDER BY year)) / LAG(AVG(total_cost)) OVER (ORDER BY year)) * 100, 2) AS percent_change
FROM costs
GROUP BY year;

   sqlite:///'costs.db'
   sqlite:///'jupyter_sqlite.db'
 * sqlite:///my_jupyter.db
Done.


year,avg_total_cost,percent_change
2013,18132.0,None
2014,18767.0,3.5
2015,19305.0,2.87
2016,19984.0,3.52
2017,20737.0,3.77
2018,21590.0,4.11
2019,22385.0,3.68
2020,22984.0,2.67
2021,21334.0,-7.18


<b>Question:</b> What is the trend of tuition costs at private institutions from 2013 to 2021?

In [259]:
%%sql
SELECT
    year, 
    ROUND(AVG(total_cost)) AS avg_total_cost,
    ROUND(((AVG(total_cost) - LAG(AVG(total_cost)) OVER (ORDER BY year)) / LAG(AVG(total_cost)) OVER (ORDER BY year)) * 100, 2) AS percent_change
FROM costs
WHERE type LIKE 'Pr%'
GROUP BY year;

   sqlite:///'costs.db'
   sqlite:///'jupyter_sqlite.db'
 * sqlite:///my_jupyter.db
Done.


year,avg_total_cost,percent_change
2013,32326.0,None
2014,33372.0,3.24
2015,34398.0,3.07
2016,35732.0,3.88
2017,37250.0,4.25
2018,39077.0,4.91
2019,40492.0,3.62
2020,41900.0,3.48
2021,42462.0,1.34


<b>Question:</b> What is the trend of tuition costs at public institutions from 2013 to 2021?

In [260]:
%%sql
SELECT
    year, 
    ROUND(AVG(total_cost)) AS avg_total_cost,
    ROUND(((AVG(total_cost) - LAG(AVG(total_cost)) OVER (ORDER BY year)) / LAG(AVG(total_cost)) OVER (ORDER BY year)) * 100, 2) AS percent_change
FROM costs
WHERE type LIKE 'Pu%'
GROUP BY year;

   sqlite:///'costs.db'
   sqlite:///'jupyter_sqlite.db'
 * sqlite:///my_jupyter.db
Done.


year,avg_total_cost,percent_change
2013,14548.0,None
2014,15079.0,3.65
2015,15494.0,2.75
2016,16008.0,3.32
2017,16527.0,3.24
2018,17174.0,3.92
2019,17766.0,3.45
2020,18158.0,2.21
2021,15917.0,-12.34


<b>Question:</b> What is the trend of tuition costs between 2 and 4 year public institutions from 2013 to 2021?

In [265]:
%%sql
SELECT
    year,
    length,
    ROUND(AVG(total_cost)) AS avg_total_cost,
    ROUND(((AVG(total_cost) - LAG(AVG(total_cost)) OVER (ORDER BY year)) / LAG(AVG(total_cost)) OVER (ORDER BY year)) * 100, 2) AS percent_change
FROM costs
WHERE type LIKE 'Pu%' AND length LIKE '2%'
GROUP BY year;

   sqlite:///'costs.db'
   sqlite:///'jupyter_sqlite.db'
 * sqlite:///my_jupyter.db
Done.


year,length,avg_total_cost,percent_change
2013,2-year,5478.0,None
2014,2-year,5677.0,3.62
2015,2-year,5773.0,1.7
2016,2-year,5949.0,3.04
2017,2-year,6103.0,2.6
2018,2-year,6348.0,4.01
2019,2-year,6493.0,2.29
2020,2-year,6560.0,1.03
2021,2-year,6676.0,1.76


In [266]:
%%sql
SELECT
    year,
    length,
    ROUND(AVG(total_cost)) AS avg_total_cost,
    ROUND(((AVG(total_cost) - LAG(AVG(total_cost)) OVER (ORDER BY year)) / LAG(AVG(total_cost)) OVER (ORDER BY year)) * 100, 2) AS percent_change
FROM costs
WHERE type LIKE 'Pu%' AND length LIKE '4%'
GROUP BY year;

   sqlite:///'costs.db'
   sqlite:///'jupyter_sqlite.db'
 * sqlite:///my_jupyter.db
Done.


year,length,avg_total_cost,percent_change
2013,4-year,23441.0,None
2014,4-year,24297.0,3.65
2015,4-year,25025.0,2.99
2016,4-year,25871.0,3.38
2017,4-year,26541.0,2.59
2018,4-year,27363.0,3.1
2019,4-year,28154.0,2.89
2020,4-year,28846.0,2.46
2021,4-year,24517.0,-15.01


#### Tuition by State
This phase of analysis includes a deeper dive into trends at a more granular level. It answers questions like:
* Which state has the least/most expensive undergraduate tuition?
* Which state has the least/most expensive 2 and 4 year programs?

It will also calculate rate of change YOY for individual states.

In [287]:
%%sql
SELECT state, ROUND(AVG(total_cost)) as avg_tuition
FROM costs
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10;

   sqlite:///'costs.db'
   sqlite:///'jupyter_sqlite.db'
 * sqlite:///my_jupyter.db
Done.


state,avg_tuition
Vermont,29341.0
Connecticut,27509.0
Massachusetts,27267.0
Rhode Island,26351.0
New Hampshire,26129.0
Pennsylvania,25827.0
District of Columbia,25416.0
New Jersey,25022.0
Oregon,24508.0
Maryland,24191.0


In [278]:
%%sql
SELECT state, ROUND(AVG(total_cost)) as avg_tuition
FROM costs
WHERE type LIKE 'Pu%'
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10;

   sqlite:///'costs.db'
   sqlite:///'jupyter_sqlite.db'
 * sqlite:///my_jupyter.db
Done.


state,avg_tuition
Delaware,23569.0
Vermont,23122.0
New Hampshire,21748.0
Connecticut,21047.0
Virginia,19742.0
Massachusetts,19690.0
Rhode Island,19642.0
Pennsylvania,19390.0
New Jersey,19216.0
Illinois,19062.0


In [286]:
%%sql
SELECT state, ROUND(AVG(total_cost)) as avg_tuition
FROM costs
WHERE type LIKE 'Pr%'
GROUP BY 1
ORDER BY 2
LIMIT 10;

   sqlite:///'costs.db'
   sqlite:///'jupyter_sqlite.db'
 * sqlite:///my_jupyter.db
Done.


state,avg_tuition
Idaho,13445.0
Utah,15486.0
Wyoming,16638.0
West Virginia,20836.0
North Dakota,21125.0
Arizona,21910.0
Mississippi,24640.0
Alabama,25124.0
Delaware,25680.0
Alaska,27664.0


In [289]:
%%sql
SELECT state, ROUND(AVG(total_cost)) as avg_tuition
FROM costs
WHERE type LIKE 'Pu%' AND length LIKE '2%'
GROUP BY 1
ORDER BY 2
LIMIT 10;

   sqlite:///'costs.db'
   sqlite:///'jupyter_sqlite.db'
 * sqlite:///my_jupyter.db
Done.


state,avg_tuition
Nebraska,3397.0
New Mexico,3483.0
Kansas,3826.0
Alaska,3834.0
Arkansas,3927.0
Mississippi,4025.0
Texas,4070.0
California,4279.0
Missouri,4587.0
Michigan,4899.0


In [288]:
%%sql
SELECT state, ROUND(AVG(total_cost)) as avg_tuition
FROM costs
WHERE type LIKE 'Pu%' AND length LIKE '4%'
GROUP BY 1
ORDER BY 2
LIMIT 10;

   sqlite:///'costs.db'
   sqlite:///'jupyter_sqlite.db'
 * sqlite:///my_jupyter.db
Done.


state,avg_tuition
District of Columbia,9517.0
South Dakota,17061.0
Wyoming,18141.0
North Dakota,19434.0
Utah,19833.0
New Mexico,20224.0
Oklahoma,21045.0
Florida,21292.0
Arkansas,21648.0
Mississippi,21747.0


Calculate YOY percent change in tuition across all types of institutions.

In [284]:
%%sql
SELECT
    year,
    ROUND(AVG(total_cost)) AS avg_total_cost,
    ROUND(((AVG(total_cost) - LAG(AVG(total_cost)) OVER (ORDER BY year)) / LAG(AVG(total_cost)) OVER (ORDER BY year)) * 100, 2) AS percent_change
FROM costs
WHERE state == 'Oregon'
GROUP BY year;

   sqlite:///'costs.db'
   sqlite:///'jupyter_sqlite.db'
 * sqlite:///my_jupyter.db
Done.


year,avg_total_cost,percent_change
2013,21184.0,None
2014,22038.0,4.03
2015,22710.0,3.05
2016,23717.0,4.44
2017,24723.0,4.24
2018,25830.0,4.48
2019,26467.0,2.47
2020,27415.0,3.58
2021,26490.0,-3.37


### Conclusions
<b>Tuition Trends</b></br>
The cost of undergraduate education in the United States has risen at consistent rates between about 2.5% and 4% YOY. A notable exception occurs between 2020 and 2021, where cost dropped by 7.18%.

Rates rose most aggressively among private institutions, rising at just under 3.5% per year. Even between 2020 and 2021 when the average cost among all universities dropped by 7.18%, private institutions rose by 1.34%.

Public institutions rose at an average of 1.27% YOY (3.22% if excluding 2021). Among public institutions, tuition at 2-year universities rose at an average of 2.61% YOY from 2013-2020 compared with their 4-year counterparts, which rose at an average of 3.01% during this time frame. From 2020 to 2021, public 2-year tuition rose by 1.76% (even more than private institutions), while public 4-year tuition fell by 15.01%.

<b>Tuition by State</b></br>
Vermont has the highest tuition costs among all institution types and lengths at just over \\$29,300 per year. Wyoming has the lowest at just over \\$12,300 per year. 

Among private institutions, Massachusetts has the highest average tuition costs at around \\$57,500 per year. Idaho has the lowest at just over \\$13,400 per year.

Among public 4-year institutions, Vermont has the highest average tuition at just under \\$37,000 per year, and Washington D.C. has the lowest at just over \\$9,500 per year.

Among public 2-year institutions, New Hampshire has the highest average tuition at over \\$11,300 per year, and Nebraska has the lowest at just under \\$3,400 per year.

## A Note on Further Analysis
Further analysis can be done to determine which states cost of college education is increasing or decrease most dramatically, and which states costs remain most stagnant. The scope of the project could also expand to include the cost of education worldwide.

## IGNORE

The following portion reformats the data for easier functionality in Tableau.

In [26]:
# Group the dataframe by the desired columns and sum the 'Value' column
grouped = cost_breakdown.groupby(['Year', 'State', 'Type', 'Length']).agg({'Value': 'sum'})

In [27]:
# Reset the index to turn the grouped columns into regular columns
grouped = grouped.reset_index()

In [28]:
# Pivot the 'Expense' column into 2 columns using the 'Value' column as the values
pivoted = cost_breakdown.pivot_table(index=['Year', 'State', 'Type', 'Length'], columns='Expense', values='Value')

In [29]:
# Reset the index to turn the grouped columns into regular columns
pivoted = pivoted.reset_index()

In [30]:
# Merge the two dataframes using the common columns
result = pd.merge(grouped, pivoted, on=['Year', 'State', 'Type', 'Length'])

result.head()

,Year,State,Type,Length,Value,Fees/Tuition,Room/Board
0,2013,Alabama,Private,4-year,22486,13983.0,8503.0
1,2013,Alabama,Public In-State,2-year,4048,4048.0,NaN
2,2013,Alabama,Public In-State,4-year,16546,8073.0,8473.0
3,2013,Alabama,Public Out-of-State,2-year,7736,7736.0,NaN
4,2013,Alabama,Public Out-of-State,4-year,28853,20380.0,8473.0


In [34]:
# Rename the columns to match the desired output
result = result.rename(columns={'Year': 'year', 'State': 'state', 'Type': 'type', 'Length': 'length', 'Value': 'total_cost', 'Fees/Tuition': 'tuition', 'Room/Board': 'room_board'})

result.head()

,year,state,type,length,tuition,room_board,total_cost
0,2013,Alabama,Private,4-year,13983.0,8503.0,22486
1,2013,Alabama,Public In-State,2-year,4048.0,NaN,4048
2,2013,Alabama,Public In-State,4-year,8073.0,8473.0,16546
3,2013,Alabama,Public Out-of-State,2-year,7736.0,NaN,7736
4,2013,Alabama,Public Out-of-State,4-year,20380.0,8473.0,28853


In [35]:
# Reorder the columns to match the desired output
result = result[['year', 'state', 'type', 'length', 'tuition', 'room_board', 'total_cost']]

In [36]:
result.head()

,year,state,type,length,tuition,room_board,total_cost
0,2013,Alabama,Private,4-year,13983.0,8503.0,22486
1,2013,Alabama,Public In-State,2-year,4048.0,NaN,4048
2,2013,Alabama,Public In-State,4-year,8073.0,8473.0,16546
3,2013,Alabama,Public Out-of-State,2-year,7736.0,NaN,7736
4,2013,Alabama,Public Out-of-State,4-year,20380.0,8473.0,28853


In [37]:
result['room_board'] = result['room_board'].fillna(0)

,year,state,type,length,tuition,room_board,total_cost
0,2013,Alabama,Private,4-year,13983.0,8503.0,22486
1,2013,Alabama,Public In-State,2-year,4048.0,0.0,4048
2,2013,Alabama,Public In-State,4-year,8073.0,8473.0,16546
3,2013,Alabama,Public Out-of-State,2-year,7736.0,0.0,7736
4,2013,Alabama,Public Out-of-State,4-year,20380.0,8473.0,28853


In [38]:
result.to_csv('undergrad_data.csv', index=False)